In [ ]:
from sys import path
from os import listdir

from pandas import DataFrame

path.append("../../../")
from componentes.pipeline.pipeline import Pipeline
from componentes.artefato.arquivo import Arquivo
from componentes.container.container_bronze import ContainerBronze
from componentes.container.container_silver import ContainerSilver
from componentes.container.categoria_de_armazenamento import \
    CategoriaDeArmazenamento
from utils.transformacoes_dataframe import (
    remover_espacamento_das_colunas,
    remover_colunas_unnamed)

In [ ]:
class PAICBronzeParaSilver(Pipeline):
    """
    Pipeline de dados responsável por executar o processamento de artefatos
    do PAIC de categoria bronze para silver
    """
    def _transformar_dados(self, dataframe: DataFrame) -> DataFrame:
        """
        Realiza a normalização e tratamento de dados do artefato
        de referência para a tabela PAIC
        """
        dataframe = self._remover_linhas_de_categoria(dataframe=dataframe)
        dataframe = self._criar_codigo_cnae(dataframe=dataframe)
        dataframe = self._de_para_codigo_cnae(dataframe=dataframe)
        dataframe = self._criar_categoria_empresa(dataframe=dataframe)
        dataframe = remover_espacamento_das_colunas(dataframe=dataframe)
        dataframe = self._de_para_valor(dataframe=dataframe)
        dataframe = self._remover_colunas_desnecessarias(dataframe=dataframe)
        return dataframe

    def _remover_linhas_de_categoria(self, dataframe: DataFrame) -> DataFrame:
        """
        Remove a totalização referente a totalização de cada divisão
        """
        self._logger.log_mensagem(mensagem="Removendo totalização de divisao")
        categorias = [
            "Total das empresas",
            "Empresas entre 1 e 4 de PO - total",
            "Empresas entre 5 e 29 de PO - total",
            "Empresas com 30 ou mais de PO - total"
        ]
        return (
            dataframe
            .query(expr="pessoal_ocupado_grupos_classes not in @categorias"))

    def _criar_codigo_cnae(self, dataframe: DataFrame) -> DataFrame:
        """
        Extrai o dado do código cnae da coluna pessoal_ocupado_grupos_classes
        em uma nova
        """
        self._logger.log_mensagem(mensagem="Criando coluna codigo_cnae")
        dataframe["codigo_cnae"] = (
            dataframe["pessoal_ocupado_grupos_classes"]
            .str
            .extract(pat=r"(.+?(?= Empresas))"))
        return dataframe

    def _de_para_codigo_cnae(self, dataframe: DataFrame) -> DataFrame:
        """
        As classes contidas nas pesquisas PAIC não possuem o dígito verificador
        previamente mapeado no sistema de códigos CNAE. Logo, é preciso realizar
        o de para das classes para adicionar essa referência
        """
        self._logger.log_mensagem(mensagem="Executando de para do codigo_cnae")
        de_para_codigo_cnae = {
            "41.10": "41.10-7",
            "41.20": "41.20-4",
            "42.11": "42.11-1",
            "42.12": "42.12-0",
            "42.13": "42.13-8",
            "42.21": "42.21-9",
            "42.22": "42.22-7",
            "42.23": "42.23-5",
            "42.91": "42.91-0",
            "42.92": "42.92-8",
            "42.99": "42.99-5",
            "43.11": "43.11-8",
            "43.12": "43.12-6",
            "43.13": "43.13-4",
            "43.19": "43.19-3",
            "43.21": "43.21-5",
            "43.22": "43.22-3",
            "43.29": "43.29-1",
            "43.30": "43.30-4",
            "43.91": "43.91-6",
            "43.99": "43.99-1"
        }
        dataframe["codigo_cnae"] = (
            dataframe["codigo_cnae"]
            .replace(to_replace=de_para_codigo_cnae))
        return dataframe

    def _criar_categoria_empresa(self,dataframe: DataFrame) -> DataFrame:
        """
        Extrair o dado da categoria da empresa da coluna
        pessoal_ocupado_grupos_classes em uma nova
        """
        self._logger.log_mensagem(mensagem="Criando coluna categoria_empresa")
        dataframe["categoria_empresa"] = (
            dataframe["pessoal_ocupado_grupos_classes"]
            .str
            .extract(pat=r"(Empresas.*?PO)"))
        return dataframe

    def _de_para_valor(self, dataframe: DataFrame) -> DataFrame:
        """
        Faz o de para de valores invalidos de acordo com a legenda da pesquisa
        """
        self._logger.log_mensagem(mensagem="Executando de para do valor")
        de_para_valor = {
            "-": "0",
            "..": "0",
            "...": "0",
            "X": "0"
        }
        dataframe["valor"] = (
            dataframe["valor"]
            .replace(to_replace=de_para_valor))
        return dataframe

    def _remover_colunas_desnecessarias(self, dataframe: DataFrame) -> DataFrame:
        """
        Remoção de colunas indesejadas
        """
        self._logger.log_mensagem(mensagem="Removendo colunas desnecessárias")
        dataframe = remover_colunas_unnamed(dataframe=dataframe)
        return dataframe.drop(columns=["pessoal_ocupado_grupos_classes"])

In [ ]:
nomes_dos_arquivos_bronze = [
    nome.replace(".csv", "")
    for nome
    in listdir(path=f"../../../dados/bronze/paic")
]
for nome in nomes_dos_arquivos_bronze:
    artefato_de_entrada = Arquivo(
        nome=nome,
        categoria_de_armazenamento=CategoriaDeArmazenamento.BRONZE,
        fonte="paic")
    container_de_entrada = ContainerBronze()
    artefato_de_saida = Arquivo(
        nome=nome,
        categoria_de_armazenamento=CategoriaDeArmazenamento.SILVER,
        fonte="paic")
    container_de_saida = ContainerSilver()
    pipeline = PAICBronzeParaSilver(
        artefato_de_entrada=artefato_de_entrada,
        container_de_entrada=container_de_entrada,
        artefato_de_saida=artefato_de_saida,
        container_de_saida=container_de_saida)
    pipeline.executar_etl()
    print("\n")